In [1]:
import time # 引入time
import matplotlib.pyplot as plt
import numpy as np

from keras.datasets import mnist
from keras.utils import np_utils
from dense import Dense
from convolutional import Convolutional
from maxpool2d import Maxpool2d
from reshape import Reshape
from activations import Sigmoid, Tanh, Relu, Softmax
from losses import binary_cross_entropy, binary_cross_entropy_prime
from network import train, predict, save_network, load_network

def preprocess_data(x, y, limit):
    # zero_index = np.where(y == 0)[0][:limit]
    # one_index = np.where(y == 1)[0][:limit]
    all_indices = np.hstack(list(np.where(y == x)[0][:limit] for x in range(10)))
    all_indices = np.random.permutation(all_indices)
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float32") / 255
    y = np_utils.to_categorical(y)
    y = y.reshape(len(y), 10, 1)
    return x, y

# load MNIST from server, limit to 100 images per class since we're not training on GPU
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = preprocess_data(x_train, y_train, 100)
x_valid, y_valid = x_train[900:], y_train[900:]
x_train, y_train = x_train[:900], y_train[:900]
x_test, y_test = preprocess_data(x_test, y_test, 100)

# nowTime = int(time.time()) # 取得現在時間
# struct_time = time.localtime(nowTime) # 轉換成時間元組
# timeString = time.strftime("%m%d%I%M%S", struct_time) # 將時間元組轉換成想要的字串
# print(timeString)

l2 = 1e-1
learning_rate = 3e-2

# neural network
network = [
    Convolutional((1, 28, 28), 5, 20, l2=l2),
    Maxpool2d(0, 2, 2),
    Tanh(),
    Reshape((20, 12, 12), (20 * 12 * 12, 1)),
    Dense(20 * 12 * 12, 100, l2=l2),
    Tanh(),
    Dense(100, 10, l2=l2),
    Softmax()
]

net_id = '1121071336'
load_network(network, net_id)


Using TensorFlow backend.


In [6]:
pred_res = [int(np.argmax(predict(network, x)) == np.argmax(y)) for x,y in zip(x_test, y_test)]
# acc = sum(pred_res)/len(pred_res)
# print(f"test accuracy: {acc}")

x, y = x_test[0], y_test[0]
y_pred = predict(network, x)
print(y_pred)
print(y)

[[0.0087469 ]
 [0.13206027]
 [0.05482804]
 [0.11968563]
 [0.05115404]
 [0.06693868]
 [0.04856423]
 [0.12924765]
 [0.07940086]
 [0.30937369]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]]
